Script to replicate the baseline $k$-anonymity pipeline for the student performance and student academics data sets.

In [ ]:
#base imports
import pandas as pd
import numpy as np
import math
import warnings
from os import path

#imports for anonymization pipeline
from pyarxaas import ARXaaS
from pyarxaas.privacy_models import KAnonymity
from pyarxaas import Dataset
from pyarxaas.hierarchy import IntervalHierarchyBuilder

#import models/metrics
import matplotlib.pyplot as plt
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import model_selection, metrics, pipeline, preprocessing
from sklearn.metrics import cohen_kappa_score, roc_auc_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

In [ ]:
#choose data set
mathia = False
acad_perf = False
epm = True

#set filename for math/Portuguese
student_dat = False

#choose if running baseline
baseline = False

Following are cells for each of the five data sets used

In [ ]:
#student math performance (or -por for Portuguese)
if student_dat:
    df_path = "..."
    df = pd.read_csv(df_path, delimiter = ';')
    print(len(df))
    d_g3 = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:1, 12:1, 13:1, 14:1, 15:1, 16:1, 17:1, 18:1, 19:1, 20:1} #convert into a binary outcome
    df['G3'] = df['G3'].map(d_g3)
    df['labels'] = df['G3']
    df = df.drop('G3', axis = 'columns')

    #convert categorical to numeric
    d_school = {'GP': 0, 'MS': 1}
    d_sex = {'M': 0, 'F': 1}
    d_age = {15:0, 16:1, 17:2, 18:3, 19:4, 20:5, 21:6, 22:7}
    d_address = {'U': 0, 'R': 1}
    d_famsize = {'LE3': 0, 'GT3': 0}
    d_pstatus = {'T': 0, 'A': 1}

    d_mjob = {'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4}
    d_fjob = {'teacher': 0, 'health': 1, 'services': 2, 'at_home': 3, 'other': 4}
    d_reason = {'home': 0, 'reputation': 1, 'course': 2, 'other': 3}
    d_guardian = {'mother': 0, 'father': 1, 'other': 2}

    d_schoolsup = {'yes': 0, 'no': 1}
    d_famsup = {'yes': 0, 'no': 1}
    d_paid = {'yes': 0, 'no': 1}
    d_activities = {'yes': 0, 'no': 1}
    d_nursery = {'yes': 0, 'no': 1}
    d_higher = {'yes': 0, 'no': 1}
    d_internet = {'yes': 0, 'no': 1}
    d_romantic = {'yes': 0, 'no': 1}

    #apply mappings
    df['school'] = df['school'].map(d_school)
    df['sex'] = df['sex'].map(d_sex)
    df['age'] = df['age'].map(d_age)
    df['address'] = df['address'].map(d_address)
    df['famsize'] = df['famsize'].map(d_famsize)
    df['Pstatus'] = df['Pstatus'].map(d_pstatus)

    df['Mjob'] = df['Mjob'].map(d_mjob)
    df['Fjob'] = df['Fjob'].map(d_fjob)
    df['reason'] = df['reason'].map(d_reason)
    df['guardian'] = df['guardian'].map(d_guardian)

    df['schoolsup'] = df['schoolsup'].map(d_paid)
    df['famsup'] = df['famsup'].map(d_paid)
    df['paid'] = df['paid'].map(d_paid)
    df['activities'] = df['activities'].map(d_paid)
    df['nursery'] = df['nursery'].map(d_paid)
    df['higher'] = df['higher'].map(d_paid)
    df['internet'] = df['internet'].map(d_paid)
    df['romantic'] = df['romantic'].map(d_paid)

    df = df.dropna()
    df = df.apply(pd.to_numeric)

    X = df.drop(['labels'], axis=1)
    y = df['labels']
    filtered_df = df.drop(['labels'], axis=1)

elif acad_perf:
    df_path = "..."
    df = pd.read_csv(df_path)

    X = df.drop(['participant_ids', 'labels'], axis=1)
    y = df['labels']
    filtered_df = df.drop(['participant_ids','labels'], axis=1)
    print(len(df))

elif mathia:
    df_path = "..."
    df = pd.read_csv(df_path)
    df = df.drop(['-'], axis=1)
    df = df.drop(['--'], axis=1)
    df = df[df.label != '?']  # These are uncodable (e.g., super short clip)
    df = df.rename(columns={"label": "labels"})
    for col in df.columns:
        if df[col].dtype == np.float64:
            if df[col].nunique() < 16:  # Replace with ranks starting at 0
                df[col] = df[col].rank(method='dense').sub(1).fillna(-1).astype(int)
            else:
                df[col] = pd.cut(df[col], 16, labels=False).fillna(-1).astype(int)
            rare_vals = df[col].value_counts() < len(df) * .05
            if rare_vals.sum():
                rare_mask = df[col].isin(rare_vals[rare_vals].index)
                df.loc[rare_mask, col] = df.loc[rare_mask, col].min()
    df = df.fillna(-1)
    df['labels'] = (df.labels == 'G').astype(int)
    df.reset_index(drop=True, inplace=True)  # Need to reset so it aligns with PyARXaaS results
    #df.to_csv('~/Downloads/kanon/tmp.csv', index=False)

    colsample_df = df.sample(frac=.25, axis=1, replace=False, random_state=1)
    X = colsample_df  # Random state results in user_id and labels already dropped
    y = df['labels']
    filtered_df = colsample_df

elif epm:
    df_path = '...'
    df = pd.read_csv(df_path)
    df = df.drop(df[df['session_id'] == 'session1'].index)

    d_sessionid = {'session2': 2, 'session3': 3, 'session4': 4, 'session5': 5, 'session6': 6}
    df['session_id'] = df['session_id'].map(d_sessionid)

    df = df.rename(columns={'session_grade': 'labels'})
    df['labels'] = df['labels'].apply(lambda x: 0 if x < 2.5 else 1)

    for col in df.columns:
        if df[col].dtype == np.float64:
            if df[col].nunique() < 16:  # Replace with ranks starting at 0
                df[col] = df[col].rank(method='dense').sub(1).fillna(-1).astype(int)
            else:
                df[col] = pd.cut(df[col], 16, labels=False).fillna(-1).astype(int)
            rare_vals = df[col].value_counts() < len(df) * .05
            if rare_vals.sum():
                rare_mask = df[col].isin(rare_vals[rare_vals].index)
                df.loc[rare_mask, col] = df.loc[rare_mask, col].min()
    df = df.fillna(-1)
    df.reset_index(drop=True, inplace=True)  # Need to reset so it aligns with PyARXaaS results

    X = df.drop(['student_id', 'labels'], axis = 1)
    y = df['labels']
    filtered_df = df.drop(['student_id', 'labels'], axis = 1)



#the interval anonymization pipeline initialization
arxaas = ARXaaS("http://localhost:8080")
def set_hierarchy(target_df):
    dataset = Dataset.from_pandas(target_df)
    for feature in target_df.columns:
        if feature == 'labels':
            continue
        if target_df[feature].dtype in [np.int64, np.int32]:
            # For integer columns, set up hierarchy in powers of 2
            interval_based = IntervalHierarchyBuilder()
            colmin = target_df[feature].min()
            colmax = target_df[feature].max()
            for start in range(colmin, colmax + 1, 1):
                interval_based.add_interval(start, start + 1, start)
            for power_exp in range(1, math.ceil(np.log2(colmax - colmin + .001))):
                level = interval_based.level(power_exp - 1)
                for start in range(colmin, colmax + 1, 2 ** power_exp):
                    level.add_group(2, start)  # Group together 2 groups from previous level
            interval_hierarchy = arxaas.hierarchy(interval_based, list(target_df[feature]))
        else:
            raise NotImplementedError('Feature type not handled:', target_df[feature].dtype,
                                      '(' + feature + ')')
        dataset.set_hierarchy(feature, interval_hierarchy)
    return dataset, interval_based

The following code block finds the feature importance of the data using a linear regression classifier.

In [ ]:
rf_regr = LinearRegression()
rf_regr.fit(X, y)

cv = model_selection.KFold(3, shuffle=True, random_state=11798)
rf_preds = model_selection.cross_val_predict(rf_regr, X, y, cv=cv)

perm_importance = permutation_importance(rf_regr, X, y)
importance_map = pd.Series(perm_importance.importances_mean, index=X.columns)

sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(X.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

Start ARXaaS: `docker run -p 8080:8080 navikt/arxaas`

In [ ]:
# Repeat classifier with 3-fold cross validation 10 times with different random states
mean_aucs = []
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    for k in range(1, 16):
        kanon = KAnonymity(k)
        filtered_dataset, _ = set_hierarchy(filtered_df)
        anon_result = arxaas.anonymize(filtered_dataset, [kanon])
        anon_result_df = anon_result.dataset.to_dataframe()

        recombined_X = anon_result_df.replace(['*'], -1)
        y = df['labels']

        if not baseline and k > 1:
            # Check how many columns have some variance after enforcing k-anonymity
            # This forms our "budget" for how many features we can have
            varying_cols = [c for c in recombined_X.columns if recombined_X[c].nunique() > 1]
            varying_importance = importance_map[varying_cols].sum()  # Importance for these
            print('k =', k, 'yields variance in', len(varying_cols), 'columns;',
                  round(varying_importance / importance_map.sum(), 3), 'prop. of importance')
            # Keep that many of the best ones if the varying ones are low importance
            if varying_importance < importance_map.sum() * .1:
                best_features = filtered_df.columns[sorted_idx[-len(varying_cols):]]
                best_subset_df = filtered_df[best_features].copy()
                # Anonymize each one individually first, with higher k for less-important features
                max_importance = importance_map.max()
                for col in best_features:
                    cur_importance = perm_importance.importances_mean[list(filtered_df.columns).index(col)]
                    cur_importance = max(cur_importance, .001)  # Avoid div/0 errors
                    one_col_df = best_subset_df[[col]]
                    one_col_dataset, _ = set_hierarchy(one_col_df)
                    one_col_kanon = KAnonymity(int(k * max_importance / cur_importance))
                    anon_one_col = arxaas.anonymize(one_col_dataset, [one_col_kanon])
                    anon_one_col_df = anon_one_col.dataset.to_dataframe()
                    best_subset_df[col] = anon_one_col_df[col].replace('*', -1)
                    best_subset_df[col] = best_subset_df[col].astype(filtered_df[col].dtype)
                # Re-anonymize that subset
                best_dataset, _ = set_hierarchy(best_subset_df)
                anon_best = arxaas.anonymize(best_dataset, [kanon])
                anon_best_df = anon_best.dataset.to_dataframe()
                recombined_X = anon_best_df.replace(['*'], -1)
                #recombined_X.to_csv('tmp.csv')

        # Reset column data types and restore columns/order
        col_ordered_df = filtered_df.copy()
        for col in filtered_df.columns:
            if col in recombined_X.columns:
                col_ordered_df[col] = recombined_X[col].astype(filtered_df[col].dtype)
            else:
                col_ordered_df[col] = -1

        for col in col_ordered_df:  # Remove zero-variance columns before ML
            if col_ordered_df[col].nunique() == 1:
                col_ordered_df.drop(columns=col, inplace=True)

        temp_accuracies = []
        temp_aucs = []
        for iter_num in range(10):
            print('Fitting model', iter_num + 1, 'for k =', k, end='\r')
            if mathia:
                cv = model_selection.GroupKFold(3)
                #ss = StandardScaler()
                #ss.fit(col_ordered_df)
                #col_ordered_df = ss.transform(col_ordered_df)
                groups = df['user_id']
                #solver = 'sag'
            elif epm:
                cv = model_selection.GroupKFold(3)
                groups = df['student_id']
            else:
                cv = model_selection.KFold(3, shuffle=True, random_state=iter_num + baseline)
                groups = None
                #solver = 'lbfgs'
            model = RandomForestClassifier(class_weight='balanced', random_state=iter_num + baseline, min_samples_leaf=10)
            #model = LogisticRegression(random_state=iter_num)

            preds = model_selection.cross_val_predict(model, col_ordered_df, y, cv=cv, groups=groups)
            auc = roc_auc_score(y, preds)
            temp_aucs.append(auc)

        mean_aucs.append(np.mean(temp_aucs))
        print('k=', k, 'mean auc:', mean_aucs[-1], ' ' * 20)

In [ ]:
#gentle ploting cell
plt.plot(np.arange(1, 16), mean_aucs, 'g', label='mean auc')
plt.xlabel('k')
plt.ylabel('AUC')
plt.legend()
plt.show()